In [225]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import yfinance as yf
import statsmodels.api as sm
import xlwings as xw

In [242]:
# Helpers

def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return v, correlation

In [287]:
# CMAs

data_sheet = xw.Book(r'F:\OneDrive\Investing\Factor_Investing\Papers\Factor_Investing_Using_CMAs_Data.xlsx').sheets('Sheet1')

result_sheet = xw.Book(r'F:\OneDrive\Investing\Factor_Investing\Papers\Factor_Investing_Using_CMAs_Data.xlsx').sheets('Sheet2')


df_CMAs_raw = data_sheet['A1'].expand().options(pd.DataFrame).value
df_CMAs_raw.set_index('Asset Class', inplace=True)


### 2020 CMA survey produced by Horizon Actuarial Services (HAS)

In [288]:
df_CMAs_raw

,10Y-Arith-Return,10Y-Geom-Return,20Y-Arith-Return,20Y-Geom-Return,Std Dev,1.0,2.0,3.0,4.0,5.0,...,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0
Asset Class,,,,,,,,,,,,,,,,,,,,,
US Equity-Large Cap,0.0740,0.0616,0.0836,0.0706,0.1622,1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US Equity-Small/Mid Cap,0.0876,0.0685,0.0954,0.0756,0.2022,0.89,1.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Non-US Equity-Developed,0.0833,0.0680,0.0909,0.0748,0.1805,0.84,0.76,1.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Non-US Equity-Emerging,0.1059,0.0785,0.1133,0.0842,0.2423,0.73,0.69,0.80,1.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US Corporate Bonds-Core,0.0275,0.0260,0.0374,0.0356,0.0547,0.15,0.08,0.17,0.16,1.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US Corporate Bonds-Long Duration,0.0313,0.0270,0.0411,0.0356,0.1016,0.14,0.07,0.14,0.11,0.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
US Corporate Bonds-High Yield,0.0536,0.0490,0.0614,0.0562,0.0975,0.63,0.62,0.62,0.62,0.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Non-US Debt-Developed,0.0158,0.0139,0.0253,0.0226,0.0702,0.12,0.06,0.28,0.23,0.53,...,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Non-US Debt-Emerging,0.0576,0.0516,0.0654,0.0585,0.1097,0.48,0.44,0.52,0.62,0.44,...,0.41,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Define the asset class used in the report

In [290]:
ls_asset_classes = df_CMAs_raw.index.to_list()
fi_asset_classes = ['US Equity-Large Cap',
                    'Non-US Equity-Developed',
                    'Non-US Equity-Emerging',
                    'US Corporate Bonds-Core',
                    'US Corporate Bonds-Long Duration',
                    'US Corporate Bonds-High Yield',
                    'Non-US Debt-Developed',
                    'Non-US Debt-Emerging',
                    'TIPS (Inflation-Protected)',
                    'Commodities',
                    'Hedge Funds',
                    'Real Estate',
                    'Infrastructure',
                    'Private Equity']
fi_asset_classes = df_CMAs_raw[df_CMAs_raw.index.isin(fi_asset_classes)].index.to_list()

### Extract the return and std dev matrix

In [291]:
df_CMAs_Returns = df_CMAs_raw[df_CMAs_raw.index.isin(fi_asset_classes)][['10Y-Arith-Return','10Y-Geom-Return','20Y-Arith-Return','20Y-Geom-Return','Std Dev']].copy()
df_CMAs_Returns.head()

,10Y-Arith-Return,10Y-Geom-Return,20Y-Arith-Return,20Y-Geom-Return,Std Dev
Asset Class,,,,,
US Equity-Large Cap,0.0740,0.0616,0.0836,0.0706,0.1622
Non-US Equity-Developed,0.0833,0.0680,0.0909,0.0748,0.1805
Non-US Equity-Emerging,0.1059,0.0785,0.1133,0.0842,0.2423
US Corporate Bonds-Core,0.0275,0.0260,0.0374,0.0356,0.0547
US Corporate Bonds-Long Duration,0.0313,0.0270,0.0411,0.0356,0.1016


### Extract the correlation matrix

In [298]:
df_CMAs_Corr = df_CMAs_raw.iloc[:-1 , -17:]
df_CMAs_Corr.set_axis(df_CMAs_Corr.index, axis=1, inplace=True)
df_CMAs_Corr.fillna(0.0, inplace=True)
df_CMAs_Corr = df_CMAs_Corr + df_CMAs_Corr.T - np.diag(np.diag(df_CMAs_Corr))
df_CMAs_Corr.set_axis(ls_asset_classes[:-1], axis=1, inplace=True)
df_CMAs_Corr.set_axis(ls_asset_classes[:-1], axis=0, inplace=True)
df_CMAs_Corr = df_CMAs_Corr[df_CMAs_Corr.index.isin(fi_asset_classes)][fi_asset_classes]
df_CMAs_Corr

,US Equity-Large Cap,Non-US Equity-Developed,Non-US Equity-Emerging,US Corporate Bonds-Core,US Corporate Bonds-Long Duration,US Corporate Bonds-High Yield,Non-US Debt-Developed,Non-US Debt-Emerging,TIPS (Inflation-Protected),Real Estate,Hedge Funds,Commodities,Infrastructure,Private Equity
US Equity-Large Cap,1.00,0.84,0.73,0.15,0.14,0.63,0.12,0.48,0.05,0.53,0.63,0.31,0.53,0.73
Non-US Equity-Developed,0.84,1.00,0.80,0.17,0.14,0.62,0.28,0.52,0.09,0.49,0.63,0.39,0.56,0.67
Non-US Equity-Emerging,0.73,0.80,1.00,0.16,0.11,0.62,0.23,0.62,0.13,0.44,0.61,0.42,0.51,0.59
US Corporate Bonds-Core,0.15,0.17,0.16,1.00,0.86,0.38,0.53,0.44,0.67,0.22,0.15,0.08,0.25,0.04
US Corporate Bonds-Long Duration,0.14,0.14,0.11,0.86,1.00,0.32,0.49,0.36,0.54,0.18,0.11,0.01,0.25,0.04
US Corporate Bonds-High Yield,0.63,0.62,0.62,0.38,0.32,1.00,0.24,0.62,0.27,0.46,0.53,0.35,0.53,0.51
Non-US Debt-Developed,0.12,0.28,0.23,0.53,0.49,0.24,1.00,0.41,0.45,0.19,0.13,0.21,0.28,0.07
Non-US Debt-Emerging,0.48,0.52,0.62,0.44,0.36,0.62,0.41,1.00,0.36,0.36,0.43,0.30,0.45,0.36
TIPS (Inflation-Protected),0.05,0.09,0.13,0.67,0.54,0.27,0.45,0.36,1.00,0.17,0.10,0.22,0.20,0.01
Real Estate,0.53,0.49,0.44,0.22,0.18,0.46,0.19,0.36,0.17,1.00,0.37,0.23,0.40,0.46


### Create Covariance Matrix by

$$\Sigma_a=diag(\sigma_X) \times Corr_{XX} \times diag(\sigma_X)$$

In [299]:
# Covariance Matrix
df_diag_std = np.diag(df_CMAs_Returns['Std Dev'])
df_Cov_Matrix = pd.DataFrame(df_diag_std.dot(df_CMAs_Corr).dot(df_diag_std))
df_Cov_Matrix.set_axis(fi_asset_classes, axis=1, inplace=True)
df_Cov_Matrix.set_axis(fi_asset_classes, axis=0, inplace=True)
df_Cov_Matrix

,US Equity-Large Cap,Non-US Equity-Developed,Non-US Equity-Emerging,US Corporate Bonds-Core,US Corporate Bonds-Long Duration,US Corporate Bonds-High Yield,Non-US Debt-Developed,Non-US Debt-Emerging,TIPS (Inflation-Protected),Real Estate,Hedge Funds,Commodities,Infrastructure,Private Equity
US Equity-Large Cap,0.026309,0.024593,0.028690,0.001331,0.002307,0.009963,0.001366,0.008541,0.000491,0.014477,0.008175,0.008850,0.012534,0.026037
Non-US Equity-Developed,0.024593,0.032580,0.034988,0.001678,0.002567,0.010911,0.003548,0.010296,0.000983,0.014894,0.009097,0.012390,0.014737,0.026594
Non-US Equity-Emerging,0.028690,0.034988,0.058709,0.002121,0.002708,0.014647,0.003912,0.016480,0.001906,0.017953,0.011824,0.017911,0.018017,0.031436
US Corporate Bonds-Core,0.001331,0.001678,0.002121,0.002992,0.004779,0.002027,0.002035,0.002640,0.002217,0.002027,0.000656,0.000770,0.001994,0.000481
US Corporate Bonds-Long Duration,0.002307,0.002567,0.002708,0.004779,0.010323,0.003170,0.003495,0.004012,0.003319,0.003080,0.000894,0.000179,0.003703,0.000894
US Corporate Bonds-High Yield,0.009963,0.010911,0.014647,0.002027,0.003170,0.009506,0.001643,0.006631,0.001593,0.007553,0.004134,0.006006,0.007534,0.010935
Non-US Debt-Developed,0.001366,0.003548,0.003912,0.002035,0.003495,0.001643,0.004928,0.003157,0.001911,0.002246,0.000730,0.002595,0.002866,0.001081
Non-US Debt-Emerging,0.008541,0.010296,0.016480,0.002640,0.004012,0.006631,0.003157,0.012034,0.002389,0.006650,0.003774,0.005792,0.007197,0.008684
TIPS (Inflation-Protected),0.000491,0.000983,0.001906,0.002217,0.003319,0.001593,0.001911,0.002389,0.003660,0.001732,0.000484,0.002343,0.001764,0.000133
Real Estate,0.014477,0.014894,0.017953,0.002027,0.003080,0.007553,0.002246,0.006650,0.001732,0.028359,0.004985,0.006817,0.009821,0.017034


 ### Factor-Mimicking Portfolio Definition: $\omega_{mf}$ and $\hat{\omega}_{pf}$

In [300]:
# Economic Growth Factor
# ls_egf_asset = ['US Equity - Large Cap', 'Non-US Equity - Developed', 'Non-US Equity - Emerging', 'Commodities']
# df_egf_returns = df_CMAs_Returns[df_CMAs_Returns['Asset Class'].isin(ls_egf_asset)].copy()
# df_egf_risks = df_CMAs_Risks[df_CMAs_Risks.index.isin(ls_egf_asset)][ls_egf_asset].copy()
weights = np.zeros((14,4), float)
df_public_weights = pd.DataFrame(weights,columns=['Economic Growth', 'Real Rate', 'Inflation' , 'Private Specific'])
df_public_weights.set_axis(df_Cov_Matrix.index, axis=0, inplace=True)

df_public_weights.loc['US Equity-Large Cap', 'Economic Growth'] = 0.25
df_public_weights.loc['Non-US Equity-Developed', 'Economic Growth'] = 0.20
df_public_weights.loc['Non-US Equity-Emerging', 'Economic Growth'] = 0.10
df_public_weights.loc['Commodities', 'Economic Growth'] = 0.10

df_public_weights.loc['TIPS (Inflation-Protected)','Real Rate'] = 1.65

df_public_weights.loc['US Corporate Bonds-Core','Inflation'] = -1.60
df_public_weights.loc['TIPS (Inflation-Protected)','Inflation'] = 1.60
df_public_weights.loc['Commodities','Inflation'] = 0.30

df_public_weights.loc['Real Estate','Private Specific'] = 0.4
df_public_weights.loc['Infrastructure','Private Specific'] = 0.4
df_public_weights.loc['Private Equity','Private Specific'] = 0.4

In [301]:
df_public_weights

,Economic Growth,Real Rate,Inflation,Private Specific
US Equity-Large Cap,0.25,0.00,0.0,0.0
Non-US Equity-Developed,0.20,0.00,0.0,0.0
Non-US Equity-Emerging,0.10,0.00,0.0,0.0
US Corporate Bonds-Core,0.00,0.00,-1.6,0.0
US Corporate Bonds-Long Duration,0.00,0.00,0.0,0.0
US Corporate Bonds-High Yield,0.00,0.00,0.0,0.0
Non-US Debt-Developed,0.00,0.00,0.0,0.0
Non-US Debt-Emerging,0.00,0.00,0.0,0.0
TIPS (Inflation-Protected),0.00,1.65,1.6,0.0
Real Estate,0.00,0.00,0.0,0.4


### Compute $\beta_{mf}$ as

$$\beta_{mf}=\Sigma_a \omega_{mf}(\omega_{mf}^{\prime} \Sigma_a \omega_{mf})^{-1}$$

In [312]:
df_omega_mf = df_public_weights[['Economic Growth', 'Real Rate', 'Inflation']]

df_beta_mf = df_Cov_Matrix.dot(df_omega_mf).dot(np.linalg.inv(df_omega_mf.T.dot(df_Cov_Matrix).dot(df_omega_mf)))

df_beta_mf.set_axis(['Economic Growth', 'Real Rate', 'Inflation'], axis=1, inplace=True)

round(df_beta_mf,2)

,Economic Growth,Real Rate,Inflation
US Equity-Large Cap,1.52,0.03,-0.27
Non-US Equity-Developed,1.71,0.05,-0.20
Non-US Equity-Emerging,2.10,0.09,-0.08
US Corporate Bonds-Core,0.13,0.56,-0.41
US Corporate Bonds-Long Duration,0.20,0.88,-0.72
US Corporate Bonds-High Yield,0.66,0.28,-0.20
Non-US Debt-Developed,0.16,0.38,-0.17
Non-US Debt-Emerging,0.62,0.44,-0.24
TIPS (Inflation-Protected),-0.00,0.61,0.00
Real Estate,0.89,0.28,-0.21


### Compute the private-speicific factor-mimicking portfolio wieights $\omega_{pf}$ as

$$\omega_{pf}=\hat{\omega}_{pf}-\omega_{mf} \beta_{mf}^{\prime} \hat{\omega}_{pf}$$

In [316]:
df_hat_omega_pf = df_public_weights[['Private Specific']]

df_omega_pf = df_hat_omega_pf - df_omega_mf.dot(df_beta_mf.T).dot(df_hat_omega_pf)

df_omega_pf

,Private Specific
US Equity-Large Cap,-0.329621
Non-US Equity-Developed,-0.263697
Non-US Equity-Emerging,-0.131849
US Corporate Bonds-Core,-0.205801
US Corporate Bonds-Long Duration,0.000000
US Corporate Bonds-High Yield,0.000000
Non-US Debt-Developed,0.000000
Non-US Debt-Emerging,0.000000
TIPS (Inflation-Protected),-0.028616
Real Estate,0.400000


#### Compute $\beta_f$ as

$$\beta_{f}=\Sigma_a \omega_{f}(\omega_{f}^{\prime} \Sigma_a \omega_{f})^{-1} \circ I$$

Where

$$\omega_f:=[\omega_{mf}\omega_{pf}]$$

In [317]:
# define omega f
df_omega_f = df_omega_mf.join(df_omega_pf)
df_omega_f

,Economic Growth,Real Rate,Inflation,Private Specific
US Equity-Large Cap,0.25,0.00,0.0,-0.329621
Non-US Equity-Developed,0.20,0.00,0.0,-0.263697
Non-US Equity-Emerging,0.10,0.00,0.0,-0.131849
US Corporate Bonds-Core,0.00,0.00,-1.6,-0.205801
US Corporate Bonds-Long Duration,0.00,0.00,0.0,0.000000
US Corporate Bonds-High Yield,0.00,0.00,0.0,0.000000
Non-US Debt-Developed,0.00,0.00,0.0,0.000000
Non-US Debt-Emerging,0.00,0.00,0.0,0.000000
TIPS (Inflation-Protected),0.00,1.65,1.6,-0.028616
Real Estate,0.00,0.00,0.0,0.400000


In [302]:
# define I I_{pf} is N x 1 vector where an entry is equal 1 for all private assets and 0 otherwise
# set to 1 for Real Estate, Infrastructure and Private Equity

i = np.zeros((14,4)) + 1
i[:,3] = 0
i[(-5, -2, -1),3] = 1
i

array([[1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [318]:
# compute beta f

df_factor_covar_matrix = df_omega_f.T.dot(df_Cov_Matrix).dot(df_omega_f)

df_implied_loadings = df_Cov_Matrix.dot(df_omega_f).dot(np.linalg.inv(df_factor_covar_matrix))
df_implied_loadings = df_implied_loadings.multiply(i)
df_implied_loadings.set_axis(['Economic Growth', 'Real Rate', 'Inflation' , 'Private Specific'], axis=1, inplace=True)
round(df_implied_loadings,2)

,Economic Growth,Real Rate,Inflation,Private Specific
US Equity-Large Cap,1.52,0.03,-0.27,0.00
Non-US Equity-Developed,1.71,0.05,-0.20,-0.00
Non-US Equity-Emerging,2.10,0.09,-0.08,-0.00
US Corporate Bonds-Core,0.13,0.56,-0.41,-0.00
US Corporate Bonds-Long Duration,0.20,0.88,-0.72,0.00
US Corporate Bonds-High Yield,0.66,0.28,-0.20,0.00
Non-US Debt-Developed,0.16,0.38,-0.17,-0.00
Non-US Debt-Emerging,0.62,0.44,-0.24,0.00
TIPS (Inflation-Protected),-0.00,0.61,0.00,0.00
Real Estate,0.89,0.28,-0.21,0.86


In [278]:

result_sheet['A1'].value = df_public_weights

result_sheet['A18'].value = df_Cov_Matrix

result_sheet['B35'].value = df_diag_std

result_sheet['A52'].value = df_CMAs_Returns

### Factor volatility and correlations from factor covariance matrixes

In [304]:
# factor covariance matrix
df_factor_covar_matrix


,Economic Growth,Real Rate,Inflation,Private Specific
Economic Growth,0.010434,0.001228,0.002532,0.013606
Real Rate,0.001228,0.009965,0.004969,0.002395
Inflation,0.002532,0.004969,0.009975,0.002762
Private Specific,0.013606,0.002395,0.002762,0.029502


In [305]:
# compute std and correlation from covariance matrix

factor_std, factor_correlation_matrix = correlation_from_covariance(df_factor_covar_matrix)
factor_std = pd.DataFrame(factor_std,index=df_factor_covar_matrix.columns)

In [306]:
# volatility
factor_std

,0
Economic Growth,0.102147
Real Rate,0.099825
Inflation,0.099875
Private Specific,0.171761


In [307]:
# factor correlation matrix
factor_correlation_matrix

,Economic Growth,Real Rate,Inflation,Private Specific
Economic Growth,1.000000,0.120399,0.248211,0.775485
Real Rate,0.120399,1.000000,0.498401,0.139699
Inflation,0.248211,0.498401,1.000000,0.160984
Private Specific,0.775485,0.139699,0.160984,1.000000


### Estimating the Factor Risk Premiums

Approach 1:
$$\mu_f = \omega_f^{\prime}\mu_a$$

Approach 2:
$$\mu_{a,pub}=c_m+\beta_{mf,pub}\mu_{mf^*}+\epsilon$$
$$\mu_{a,priv}-\beta_{mf,priv}\mu_{mf^*}=c_p+\beta_{mf,priv}\mu_{pf^*}+\eta$$

In [319]:
# Risk Premium
# Approach 1
df_returns = df_omega_f.T.dot( df_CMAs_Returns[['10Y-Arith-Return']])
df_returns

,10Y-Arith-Return
Economic Growth,0.050450
Real Rate,0.035640
Inflation,0.004660
Private Specific,0.035178


In [320]:
df_returns = df_omega_f.T.dot( df_CMAs_Returns[['10Y-Geom-Return']])
df_returns

,10Y-Geom-Return
Economic Growth,0.040040
Real Rate,0.032670
Inflation,-0.000350
Private Specific,0.029601


In [321]:
df_returns = df_omega_f.T.dot( df_CMAs_Returns[['20Y-Arith-Return']])
df_returns

,20Y-Arith-Return
Economic Growth,0.056010
Real Rate,0.048510
Inflation,0.004000
Private Specific,0.035374


In [322]:
df_returns = df_omega_f.T.dot( df_CMAs_Returns[['20Y-Geom-Return']])
df_returns

,20Y-Geom-Return
Economic Growth,0.045070
Real Rate,0.045045
Inflation,-0.001160
Private Specific,0.029067


In [ ]:
# Approach 2

## From Factors to Assets

### Optimizing the asset portfolio also repects the target factor exposure $\overline{w}_F$

$$\underset{w}{\argmin}
\gamma
(w^\prime\beta_f-\overline{w}^\prime_F)
(w^\prime\beta_f-\overline{w}^\prime_F)^\prime
+
(1-\gamma)
(w-\overline{w}_A)^\prime
(w-\overline{w}_A)$$

Where $\overline{w}_F$ is the desired(target) set of factor exposures, $\overline{w}_A$ is the target asset weights

##### Closed-form solution:

$$w=[\gamma\beta_f\beta^\prime_f+(1-\gamma)I_{N\times N}]^{-1}
(\gamma \beta_f \overline{w}_F
+
(1-\gamma)\overline{w}_A
)
$$

##### Let $\overline{w}_F$ be the mean-variance tangency factor portfolio and $\overline{w}_A$ to be the inverse volatility asset portfolio:

$$w=[\gamma\beta_f\beta^\prime_f
+(1-\gamma)I_{N\times N}]^{-1}
\bigg\lgroup
\gamma\beta_f{{\sum^{-1}_f \mu_f}\over{\mid \bold{1}^\prime_M \sum^{-1}_f \mu_f \mid}}
+(1-\gamma){{D^{-1}\bold{1}_N}\over{\mid \bold{1}^\prime_N D^{-1}\bold{1}_N\mid}}
\bigg\rgroup$$

Where $D$ is a diagonal matrix with the asset volatilites along its diagonal

##### Notes
Choosing inverse volatility approach over other risk-based approaches:
1. it avoids negative allocations to asset classes (negative weights could be undesirable for asset owners)
2. it enforces risk diversification across asset class line items
3. it is marginally affected by estimation error because it relies solely on volatility estimates, which are considerably more reliable than correlation or expected returns estimates